# Exercise 1: Basic LeanIX AI Agent

## Goal

We want you to make yourself familiar with LeanIX and to learn how you can find information about Enterprise Applications of your company both in the LeanIX workspace as well as via an AI Agent based on MCP.

## Context

New LeanIX customers typically start their LeanIX journey with [Application Rationalization](https://www.leanix.net/en/wiki/apm/application-rationalization) use case. They assess their Enterprise applications in order to make sure they align with business goals. LeanIX offers two metrics for this: [Functional fit and technical fit](https://help.sap.com/docs/leanix/ea/application-portfolio-assessment-enrich-data?locale=en-US).

- Functional fit: Does this tool do what the business needs it to do?
- Technical fit: Can we run and support this tool effectively in our IT environment? 

During this exercise, we want to identify applications that fit our business goals best. 

We thus formulate the following question: 

**Which applications have a perfect functional fit and fully appropriate technical fit?**


## Tasks

Your task is to answer the above question: 
- first in LeanIX directly
- later with an AI Agent via MCP

We will go through these steps:

1. Find the information in LeanIX
1. Create LeanIX API token
1. Use the token to connect to the LeanIX MCP Server
1. Filter the MCP tools 
1. Connect to an LLM on AI Core
1. Create an AI Agent

**Let's get started!**

## Task 1.1: Find information in LeanIX

In order to answer the above question in LeanIX, we will navigate to the workspace and use a pre-built report to find the applications that match the defined criteria.

**🚧 Your task:** Please follow these steps:

1. Log in to your [LeanIX demo workspace](https://demo-eu-9.leanix.net/SAPTechED/)
1. Navigate to Reports and select Application Portfolio report
1. Clear the filters for lifecycle and quality seal
1. Hover over the green bubble in the upper right corner
1. Click on the green bubble in the upper right corner
1. Click on "Show in Inventory"

You should now see the list of your "rock star" applications - those who have the best possible functional and technical fit.

### ✅ Solution
<details>
<summary>Click to expand solution</summary>

![Screenshot portfolio report](../../images/portfolio-report.png)

10 applications match these criteria

</details>

## Task 1.2: Create LeanIX API token

In order for us to access the same information via MCP, we need to create an API token. 

**🚧 Your task:** Please follow these steps:

1. In LeanIX, click on the user initials in the upper right corner
1. Select Administration
1. Scroll down to the section "Discovery and Integrations" and select "Technical Users"
1. Click on "Create Technical User" or "New Technical User"
1. As a name type in `mcp` + your assigned user number in the session
    - e.g., if your user number if "001" --> type "mcp001"
1. For permission role select "MEMBER"
1. Set the expiry date to the last day of the current month
1. Hit Save 
1. **Copy the Access token!** - only then click OK
1. Go back to VSCode and paste the access token to your [`.env` file](../../.env) and assign it to the environment variable `LEANIX_API_TOKEN`

Your technical user configuration should look similar to this:

![Technical user](../../images/technical-user.png)


## Task 1.3: Connect to LeanIX MCP Server

Let's connect to the LeanIX MCP Server. We will use the API token from the previous step. 

#### Load API token

Let's first load the API token from the environment.

**🚧 Your task:** Execute the cell below.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()


LEANIX_API_TOKEN = os.getenv("LEANIX_API_TOKEN")

if LEANIX_API_TOKEN:
    print("LEANIX_API_TOKEN loaded successfully.")
else:
    print("Warning: LEANIX_API_TOKEN is not set or empty.")

You should see a message "loaded successfully".

#### Connect to MCP Server

Now connect to the LeanIX MCP Server. The code below creates an MCP Client using the LangChain library to connect and get the tools. 

**🚧 Your task:** Execute the cell below.

In [ ]:
import base64
import pandas as pd
from IPython.display import display, HTML
from langchain_mcp_adapters.client import MultiServerMCPClient

lx_api_token_base64 = base64.b64encode(
    f"apitoken:{LEANIX_API_TOKEN}".encode()
).decode()

client = MultiServerMCPClient(
    {
        "LeanIX MCP Remote": {
            "transport": "streamable_http",
            "url": "https://demo-eu-2.leanix.net/services/mcp-server/v1/mcp",
            "headers": {"Authorization": f"Basic {lx_api_token_base64}"},
        }
        # Add more servers as needed
    }
)

tools = await client.get_tools()

tool_data = [{
    'Name': tool.name,
    'Description': tool.description
} for tool in tools]

# Create DataFrame from tool data
df_tools = pd.DataFrame(tool_data)

# Truncate Description column to 200 characters
df_tools['Description'] = df_tools['Description'].apply(lambda x: x[:200] + ('...' if len(x) > 200 else ''))

# Show all text in columns and align left for readability
pd.set_option('display.max_colwidth', None)
styles = [
    {"selector": "th", "props": [("text-align", "left")]},
    {"selector": "td", "props": [("text-align", "left")]}
]

# Display the table
display(HTML(df_tools.style.set_table_styles(styles).to_html()))

You should now see a table with the tools available on the MCP Server.

*The code connects to the LeanIX MCP Server using an API token, retrieves a list of available tools, and displays them in a formatted table. It first encodes the API token in base64, sets up a client with the necessary headers, and asynchronously fetches the tools. The code then creates a pandas DataFrame from the tool data, truncates long descriptions for readability, and uses custom styles to left-align the table columns. Finally, it displays the styled table as HTML in the notebook for easy viewing.*

## Task 1.4: Filter the MCP Tools

It is good practice to do some client-side filtering of tools as too many tools can overwhelm an LLM. We will do very basic filtering based on keywords for the moment to demonstrate the principle. In a real production system, we would rather implement semantic search over tool descriptions like in [rag-mcp](https://github.com/fintools-ai/rag-mcp).

**🚧 Your task:** Execute the cell below.

In [ ]:
# print the first tool from the list for reference
print("First tool details:")
print(tools[0])

Now you know the format of the `tool` data structure. Use this knowledge to filter by tool description.

**🚧 Your task:**

Create a list `filtered_tools` which contains only the tools that have the word "application" in their description. 

In [ ]:
filtered_tools = [
    # Your code here
]

print(f"Filtered tools ({len(filtered_tools)}):")
for tool in filtered_tools:
    print(f"Tool: {tool.name}")
    print(tool.description)
    print("------")

You should now see 3 tools in `filtered_tools`. The selection is not perfect but good enough for us to continue as the LLM will figure out the right tool to call later on.

### ✅ Solution
<details>
<summary>Click to expand solution</summary>

```python
filtered_tools = [
    t for t in tools
    if "application" in (t.description or "").lower() 
]

print(f"Filtered tools ({len(filtered_tools)}):")
for tool in filtered_tools:
    print(f"Tool: {tool.name}")
    print(tool)
    print("------")
```

</details>

## Task 1.5: Connect to an LLM via AI Core

Each MCP-based scenario always involves three players: 
- One or multiple MCP Servers 
- A Hosting app which connects to the MCP Servers and runs the AI Agents
- An LLM which makes the tooling decisions and generates the final answers to user requests

We will now add the LLM to the game in this task.

#### Initialize AI Core and connect to LLM

We start with loading the AI Core credentials and connecting to an LLM.

**🚧 Your task:** Check that all environment variables `AICORE_xxx` mentioned below are set in the [.env file](../../.env), then execute the cell below.

In [ ]:
from gen_ai_hub.proxy.langchain import init_llm

AICORE_CLIENT_ID = os.getenv("AICORE_CLIENT_ID")
AICORE_CLIENT_SECRET = os.getenv("AICORE_CLIENT_SECRET")
AICORE_RESOURCE_GROUP = os.getenv("AICORE_RESOURCE_GROUP")
AICORE_BASE_URL = f"{os.getenv('AICORE_BASE_URL')}/v2/lm"
AICORE_AUTH_URL = f"{os.getenv('AICORE_AUTH_URL')}/oauth/token"

# Check if all variables are set (non-empty)
required_vars = [
    ('AICORE_CLIENT_ID', AICORE_CLIENT_ID),
    ('AICORE_CLIENT_SECRET', AICORE_CLIENT_SECRET),
    ('AICORE_RESOURCE_GROUP', AICORE_RESOURCE_GROUP),
    ('AICORE_BASE_URL', AICORE_BASE_URL),
    ('AICORE_AUTH_URL', AICORE_BASE_URL),
]
missing = [name for name, val in required_vars if not val]
if missing:
    print(f"Warning: The following AI Core environment variables are not set: {', '.join(missing)}")
else:
    print("All required AI Core environment variables are set.")

# Connect to the LLM
llm = init_llm("gpt-4.1", max_tokens=32767)

# Structured print of important LLM fields
fields = [
    ('model_name', getattr(llm, 'model_name', None)),
    ('max_tokens', getattr(llm, 'max_tokens', None)),
    ('temperature', getattr(llm, 'temperature', None)),
]
print("LLM Configuration")
for name, value in fields:
    if value is not None:
        print(f"  {name}: {value}")

You should see confirmation that all environment variables are set and the configuration of the LLM we connected to.

#### Test the LLM

We now need to test if the LLM works as expected.

**🚧 Your task:** Define a test prompt. Then execute the cell. 

Bonus task: Extract token usage from the response metadata. We added some debug output to make this task easier.

In [ ]:
from IPython.display import Markdown, display
import pprint

# TODO: 🚧 Write a test prompt of your choice - you can write anything you want
test_prompt = ""
response = llm.invoke(test_prompt)

# Print the output as Markdown
display(Markdown(f"**LLM Response:**\n\n{response.content}"))

# Debug print the full response object to understand its structure
pprint.pprint("Use this structure to find the right total tokens field")
pprint.pprint(f"debug string: {response}")

# TODO: Bonus 🚧 Extract the number of tokens used from the response metadata
# total_tokens = 
# display(Markdown(f"`Total tokens used: {total_tokens}`"))

### ✅ Solution

<details>
<summary>Click to expand solution</summary>

```python
from IPython.display import Markdown, display

response = llm.invoke("Hello world")

# Print the main response content as Markdown
display(Markdown(f"**LLM Response:**\n\n{response.content}"))

# Print the token usage in a highlighted way
total_tokens = response.response_metadata["token_usage"]["total_tokens"]
display(Markdown(f"`Total tokens used: {total_tokens}`"))
```

</details>

## Task 1.6: Create AI Agent

Now we want to tackle the 3rd player of the MCP scenario and create an AI Agent that exposes MCP tools to an LLM to answer questions. We use a ReAct Agent from the LangGraph library for this purpose.


#### Understanding ReAct Agents

A **ReAct agent** (Reason + Act) is an AI agent that can both reason about problems and take actions using external tools. In LangGraph, a ReAct agent can:

- Think step-by-step about a user's question
- Decide when to use a tool (like searching in LeanIX)
- Combine information from multiple sources
- Respond in a way that's easy for employees to understand

The agent acts as a knowledgeable guide that knows how to look up information and explain it clearly to internal users.

#### Initialize AI Agent

We define the a LangGraph ReAct Agent with the cell below. 

**🚧 Your task:** Run the cell below to create the Agent.

In [ ]:
from langgraph.prebuilt import create_react_agent
from IPython.display import Markdown, display, Image 

agent =  create_react_agent(llm, filtered_tools)
print(f"Initialized Agent: {agent}")

async def call_agent(agent, query):
    response = await agent.ainvoke(
        {
            "messages": [
                {"role": "user", "content": query}
            ]
        }
    )
    return response

def print_agent_response(response):
    msgs = response.get("messages", [])
    for msg in reversed(msgs):
        content = getattr(msg, "content", "")
        if content:
            display(Markdown(content))
            break
    else:
        print("No printable content in agent response.")


display(Image(agent.get_graph().draw_mermaid_png()))


You can see the structure of the Agent in the graph. It's a simple loop deciding on tools and calling tools until the Agent thinks it has reached its goal.

#### Call the Agent

Now we want the Agent to answer our Enterprise Architecture question. 

**🚧 Your task:** Change the question below to the question we defined in the beginning of this exercise. Then execute the cell.

In [ ]:
user_question = "What is the weather like today?"

response = await call_agent(
    agent, 
    user_question
)
print_agent_response(response)

You should now see the same list of 10 applications that we got in the LeanIX application in Task 1.1.

You can further play around with other questions, also trying to be less precise with your ask and see if the AI Agent is able to handle it.

### ✅ Solution
<details>
<summary>Click to expand solution</summary>

```python
response = await call_agent(
    agent, 
    "Which applications have a perfect functional fit and fully appropriate technical fit?"
)
print_agent_response(response)
```

</details>

Congratulations! You have successfully mastered this exercise.